# Coarray Fortran Kernel

This jupyter notebook Fortran kernel uses the following software stack:
 - [GFortran](https://gcc.gnu.org/wiki/GFortran)
 - OpenCoarrays [Web](http://www.opencoarrays.org) [Github](https://github.com/sourceryinstitute/OpenCoarrays)
 - [MPICH](https://www.mpich.org)
 - [jupyter-CAF-kernel](https://github.com/sourceryinstitute/jupyter-CAF-kernel)
 
## Software versions:
 
Please see the output below for the current version of each component of the software stack

In [1]:
%num_images: 1
program get_mpirun_version
    implicit none
    character(len=90000) :: str_buff
    call execute_command_line ("gfortran --version; caf --version; mpirun --version", wait=.true., cmdmsg=str_buff)
    print*, trim(str_buff)
end program

GNU Fortran (GCC) 7.1.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.


OpenCoarrays Coarray Fortran Compiler Wrapper (caf version 1.9.0)
Copyright (C) 2015-2016 Sourcery, Inc.

OpenCoarrays comes with NO WARRANTY, to the extent permitted by law.
You may redistribute copies of OpenCoarrays under the terms of the
BSD 3-Clause License.  For more information about these matters, see
the file named LICENSE.

HYDRA build details:
    Version:                                 3.2
    Release Date:                            Wed Nov 11 22:06:48 CST 2015
    CC:                              gcc    
    CXX:                             no    
    F77:                             gfortran   
    F90:                             gfortran   
    Configure options:                       '--disable-option-checking' '--prefix=/usr/local' '--disable-

## Getting started

Each Coarray Fortran cell must be the contents a compilable Fortran source code file producing a program. The exception to this are the "magics" discussed below, which are lines starting with the percent character, `%`, that controll the compilation and execution of the program. Additional program units, e.g., modules, may be specified ahead of the main program. Only one program per cell is allowed.

### Controlling parallel execution

By default any Fortran code will be launch with the `cafrun` wrapper script, which will hanle calls to the Coarray implementation's backend, in this case [MPICH](https://www.mpich.org/)'s `mpirun`. Since this notebook executes in the cloud on one puny docker container, the __default number of images, `num_images`, has been set to 4__. *Use of more than 10 images __could CRASH__ your jupyter notebook if run in the cloud using [Binder].*

To __control the number of images__ we have implemented a jupyter notebook "magic" for the form:

__`%num_images: <n>`__  
where `<n>` should be replaced by a suitable (read small) positive integer.

### Controlling compilation

Additional "magics" have been implemented to control the compilation of the program. These are as follows:

 - __`%fcflags: <flag1>[ <flag2> [ ... [<flagN>]]]`__  
   Use this magic to pass flags to the fortran compiler, [GFortran](https://gcc.gnu.org/wiki/GFortran), via the `caf` compiler wrapper script. The `caf` script ensures propper compilation linking against [OpenCoarrays](https://github.com/sourceryinstitute/OpenCoarrays).
 - __`%ldflags: <linkerflag1>[ <linkerflag2> [ ... [<linkerflagN>]]]`__  
   Specify flags to pass to the linker. This should rarely be necessary.

### Controlling arguments passed to compiled programs

The __`%args: <arg1>[ <arg2> [ ... [<argN>]]]`__ "magic" may be used to pass command line arguments to your program.

## A brief example

An example program demonstrating most of these "magics" is shown below:

[Binder]: https://beta.mybinder.org

In [2]:
!! The following lines are Coarray Fortran jupyter notebook "magics".
!! They are stripped before sending the code to the compiler

! Control how many coarray images we spawn:
%num_images: 5

! Pass command line arguments to our program:
%args: IamOne IamTwo IamThree

! Use the following extra compilation flags:
%fcflags: -fbacktrace -std=f2008 -W -Wall -Wextra -pedantic

!! Begin the main program
program main
    implicit none

    integer :: istat, num_arg
    character(len=32), allocatable :: arg(:)
    
    associate(me => this_image()) ! Use an associate construct to avoid repitition
        ! A simple "Hellow (Coarray) world"
        write(*,'(2(A,I0),A)') "Hello from image ", me, " of ", num_images(), "!"
        
        ! Have each image process one command line argument
        num_arg = command_argument_count()
        allocate(arg(num_arg))
        if (me <= num_arg) then
            call get_command_argument (me, value=arg(me), status=istat)
            select case(istat)
            case(0)
                write(*,'(A,I0,A)') "Argument number ", me, " was: "//trim(arg(me))
            case default
                write(*,'(A,I0,A)') "Image ", me, " encountered an unknown problem retrieving it's command line argument."
            end select
        end if
    end associate
end program

Hello from image 5 of 5!
Hello from image 1 of 5!
Hello from image 4 of 5!
Argument number 1 was: IamOne
Hello from image 3 of 5!
Argument number 3 was: IamThree
Hello from image 2 of 5!
Argument number 2 was: IamTwo
